# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
    # join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

    # reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile:
        
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        # skip header row
        next(csvreader)
        
         # extracting each data row one by one and append it        
        for line in csvreader:
            # print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
# print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


8056


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### A Cassandra `Cluster` instance
is setup in the local machine by passing a single element list to the `contact_points=` Cluster parameter.  
A `Session` is then obtained from the `Cluster` instance.

In [5]:
from cassandra.cluster import Cluster
cluster = Cluster(contact_points=['127.0.0.1'])

session = cluster.connect()

#### the `udacity` Keyspace is created

In [6]:
try:
    session.execute("""
        CREATE KEYSPACE IF NOT EXISTS
            udacity
            
        WITH REPLICATION = {
             'class': 'SimpleStrategy'
            ,'replication_factor': 1
        }
    """)
    
except Exception as exception:
    print(exception)

#### the `udacity` Keyspace
is set as default for this Session

In [7]:
try:
    session.set_keyspace('udacity')
    
except Exception as exception:
    print(exception)

### Now we need to create tables to run the following queries. 
> Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## QUESTION: create queries to ask the following three questions of the data

 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




### ANSWER: for each of the queries above, a table will be created:

* Query 1 → the `session_activities` table will be modeled to answer this question;
* Query 2 → will be answered by the `user_activities` table;
* Query 3 → the `song_audience` table will be used to answer this question;  
  
First, two functions to execute CQL statements are defined.  
  
Then, all `DROP TABLE` and `CREATE TABLE` statements are declared in the next two cells

In [8]:
def execute_statements(statement_list):
    """ Sequentially executes the list of CQL statements received."""
    
    message = "Statements execution succesfully completed."
    
    for statement in statement_list:
            
        try:
            session.execute(statement)
        except Exception as exception:
            print(exception)
            message = "Statement execution wasn't successful"
            
    print(message)
    
def select_and_exhibit_cassandra_data(select_statement):
    """ Performs CQL SELECT statements and print its results."""
    
    try:
        rows = session.execute(select_statement)       
    except Exception as exception:
        print(exception)

    for row in rows:
        print(row)
        

#### `DROP TABLE` statements are set
to clear keyspace objects when necessary

In [9]:

drop_table_session_activities = """
    DROP TABLE IF EXISTS
        udacity.session_activities
    ;
"""

drop_table_user_activities = """
    DROP TABLE IF EXISTS
        udacity.user_activities
    ;
"""

drop_table_song_audience = """
    DROP TABLE IF EXISTS
        udacity.song_audience
    ;
"""

all_drop_table_statements = list((drop_table_session_activities, drop_table_user_activities, drop_table_song_audience))

# a function is called to execute all DROP TABLE statements
execute_statements(all_drop_table_statements)

Statements execution succesfully completed.


#### `CREATE TABLE` statements
are declared for the three tables involved in this exercise. All tables are then created at once.  
  
**Modeling rationale:**  
  
* `session_acitivities (query 1)` **COMPOSITE PRIMARY KEY made of** → PARTITION KEY set to "session_id" and "item_in_session" as they are present in the `SELECT` statement's `WHERE` clause: they're the criteria used to filter our results. CLUSTERING COLUMNS were set to "artist_name" and "song_title" to sort the data according to these two columns.  
  
  
* `user_activities (query 2)` **COMPOSITE PRIMARY KEY made of** → PARTITION KEY set to "user_id" and "session_id" because they're criteria to retrieve our results, used in the `SELECT` statement's `WHERE` clause. CLUSTERING COLUMN was set to "item_in_session" to sort session events in ascending order.  
  
  
* `song_audience (query 3)` **COMPOSITE PRIMARY KEY made of** → PARTITION KEY set to "song_title" as this is the attribute we'll be filtering to obtain results. "user_id" has been set to CLUSTERING COLUMN not only for the partition data to be sorted upon it, but also to make sure each row has a unique PRIMARY KEY.

In [10]:

create_table_session_activites = """
    CREATE TABLE IF NOT EXISTS
        udacity.session_activities(        
             session_id INT
            ,item_in_session INT
            ,artist_name TEXT
            ,song_title TEXT
            ,song_length FLOAT        
            
            ,PRIMARY KEY(
                 (session_id, item_in_session)
                ,artist_name
                ,song_title
                )
        )
    ;
"""

create_table_user_activities = """
    CREATE TABLE IF NOT EXISTS
        udacity.user_activities(
             user_id INT
            ,session_id INT
            ,item_in_session INT
            ,user_first_name TEXT
            ,user_last_name TEXT
            ,artist_name TEXT
            ,song_title TEXT
            
            ,PRIMARY KEY(
                 (user_id, session_id)
                ,item_in_session
            )
        )
    ;
"""

create_table_song_audience = """
    CREATE TABLE IF NOT EXISTS
        udacity.song_audience(
                 song_title TEXT
                ,user_id INT
                ,user_first_name TEXT
                ,user_last_name TEXT
                
                ,PRIMARY KEY(song_title, user_id)
            )
    ;
"""

all_create_table_statements = list((create_table_session_activites, create_table_user_activities, create_table_song_audience))

# a function is called to execute all CREATE TABLE statements
execute_statements(all_create_table_statements)      

Statements execution succesfully completed.


### **QUERY 1** 
#### the `INSERT INTO` statement
is created and executed for the `session_activities` table

In [11]:
insert_into_session_activities = """
    INSERT INTO
        udacity.session_activities(
             session_id
            ,item_in_session
            ,artist_name
            ,song_title
            ,song_length
        )
    VALUES(
         %s
        ,%s
        ,%s
        ,%s
        ,%s
    );
"""

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:

        session.execute(insert_into_session_activities, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### a `SELECT` statement is performed
to answer the first question.

In [12]:
select_session_activities = """
    SELECT
         artist_name
        ,song_title
        ,song_length
    FROM
        udacity.session_activities
    WHERE
        session_id = 338
        AND
        item_in_session = 4
    ;        
"""

# the UDF to retrieve data is called
select_and_exhibit_cassandra_data(select_session_activities)

Row(artist_name='Faithless', song_title='Music Matters (Mark Knight Dub)', song_length=495.30731201171875)


### QUERY 2 
#### `INSERT INTO user_activities` statement
is declared and performed below:

In [13]:

insert_into_user_activities = """
    INSERT INTO
        udacity.user_activities(
             user_id
            ,session_id
            ,item_in_session
            ,user_first_name
            ,user_last_name
            ,artist_name
            ,song_title
        )
    VALUES(
         %s
        ,%s
        ,%s
        ,%s
        ,%s
        ,%s
        ,%s
    );
"""

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:

        session.execute(insert_into_user_activities, ( int(line[10])
                                                      ,int(line[8])
                                                      ,int(line[3])
                                                      ,line[1]
                                                      ,line[4]
                                                      ,line[0]
                                                      ,line[9]
                                                     )
                       )

#### the second question
is then answered below:  
> the `item_in_session` attribute's been added to explicitly show the sequence of events during the User's session

In [14]:
select_user_activities = """
    SELECT
         artist_name
        ,item_in_session
        ,song_title
        ,user_first_name
        ,user_last_name
    FROM
        udacity.user_activities
    WHERE
        user_id = 10
        AND
        session_id = 182
    ;
"""

# the UDF to retrieve data is called
select_and_exhibit_cassandra_data(select_user_activities)                  

Row(artist_name='Down To The Bone', item_in_session=0, song_title="Keep On Keepin' On", user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Three Drives', item_in_session=1, song_title='Greece 2000', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Sebastien Tellier', item_in_session=2, song_title='Kilometer', user_first_name='Sylvie', user_last_name='Cruz')
Row(artist_name='Lonnie Gordon', item_in_session=3, song_title='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', user_first_name='Sylvie', user_last_name='Cruz')


### QUERY 3
#### `INSERT INTO song_audience` table
is declared and performed below:

In [15]:

insert_into_song_audience = """
    INSERT INTO
        udacity.song_audience(
             song_title
            ,user_id
            ,user_first_name
            ,user_last_name
        )
    VALUES(
         %s
        ,%s
        ,%s
        ,%s
    );
"""

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    
    for line in csvreader:

        session.execute(insert_into_song_audience, (line[9], int(line[10]), line[1], line[4]))

#### the third query
is then answered below:

In [16]:

select_song_audience = """
    SELECT
         user_first_name
        ,user_last_name
    FROM
        udacity.song_audience
    WHERE
        song_title = 'All Hands Against His Own'
    ;
"""

# the UDF to retrieve data is called
select_and_exhibit_cassandra_data(select_song_audience)

Row(user_first_name='Jacqueline', user_last_name='Lynch')
Row(user_first_name='Tegan', user_last_name='Levine')
Row(user_first_name='Sara', user_last_name='Johnson')


### Drop the tables before closing out the sessions

In [17]:
# the function for sequential statement execution is called once more
execute_statements(all_drop_table_statements)

Statements execution succesfully completed.


### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()